In [1]:
import os

os.environ["HF_HOME"] = "/projects/bhuang/.cache/huggingface"
# os.environ["OMP_NUM_THREADS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
import numpy as np
from datasets import load_dataset, Dataset

/home/bhuang/anaconda3/envs/ocd-10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_files = [
    "/home/bhuang/icd_10/data/quaero_icd10_by_category_resplitted/drbenchmark_quaero-medline-train-cls-mistral_large_instruct_2407-processed.jsonl",
    "/home/bhuang/icd_10/data/quaero_icd10_by_category_resplitted/drbenchmark_quaero-emea-train-cls-mistral_large_instruct_2407-processed.jsonl",
]

dataset = load_dataset("json", data_files=data_files, split="train")
# dataset = dataset.remove_columns(["gen_configs"])
dataset

Dataset({
    features: ['text', 'has_diso', 'labels'],
    num_rows: 1530
})

In [4]:
dataset.filter(lambda x: x["labels"], num_proc=32)

Dataset({
    features: ['text', 'has_diso', 'labels'],
    num_rows: 1530
})

In [14]:
# flatten
total_labels = sum(dataset["labels"], [])
# overall label frequecies
label_freqs = dict(zip(*np.unique(total_labels, return_counts=True)))

target_freq = max(label_freqs.values())
# target_freq = 5

In [15]:
# todo: other upsampmling methods other than duplication: add noise words, randomly delete words (tf-idf), paraphrasing, back translation

data_df = dataset.to_pandas()
data_df["labels"] = data_df["labels"].map(lambda x: x.tolist())
data_lst = data_df.to_dict("records")
# len(data_lst)

upsampled_data_lst = []
for example in data_lst:
    labels = example["labels"]
    if labels:
        min_freq = min(label_freqs[l] for l in labels)
        multiplier = max(int(target_freq / min_freq), 1)
        upsampled_data_lst.extend([example] * multiplier)
    else:
        upsampled_data_lst.append(example)

upsampled_dataset = Dataset.from_list(upsampled_data_lst)
upsampled_dataset

Dataset({
    features: ['text', 'has_diso', 'labels'],
    num_rows: 35301
})

In [16]:
# output_file = "/home/bhuang/icd_10/data/quaero_icd10_by_category_resplitted/drbenchmark_quaero-emea_medline-train-upsampled.jsonl"
output_file = f"/home/bhuang/icd_10/data/quaero_icd10_by_category_resplitted/drbenchmark_quaero-emea_medline-train-upsampled{target_freq}.jsonl"

upsampled_dataset.to_json(output_file, orient="records", lines=True, force_ascii=False)

Creating json from Arrow format: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 341.31ba/s]


5393784